# Chapter 13: Loading and Preprocessing Data with TensorFlow

Bab ini berfokus pada cara menangani dataset besar yang tidak muat di memori (RAM) menggunakan **Data API** (`tf.data`). Kita akan belajar membangun *input pipeline* yang efisien agar GPU/TPU tidak menunggu data dari CPU.

## Daftar Isi:
1. **The Data API**: Dasar-dasar objek `Dataset`.
2. **Transformasi Data**: Chaining, Shuffling, Batching, dan Prefetching.
3. **Membaca CSV Skala Besar**: Menggunakan `interleave` untuk file paralel.
4. **Format TFRecord**: Menggunakan format biner efisien TensorFlow.
5. **Keras Preprocessing Layers**: Normalisasi dan Encoding di dalam Model.

## 1. Dasar-Dasar Data API

API ini memungkinkan kita membaca data dari memori atau disk dengan cara yang terstandardisasi.

In [ ]:
import tensorflow as tf
import numpy as np

# Membuat dataset dari Python list
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

# Menampilkan isi dataset
for item in dataset:
    print(item.numpy())

## 2. Rantai Transformasi (Chaining Transformations)

Kita bisa memodifikasi data secara bertahap menggunakan metode *chaining*.

In [ ]:
# Mengulang dataset 3 kali dan mengelompokkan dalam batch berisi 7
dataset = tf.data.Dataset.range(10)
dataset = dataset.repeat(3).batch(7)

for item in dataset:
    print("Batch:", item.numpy())

### Map, Filter, dan Shuffle

- **`map`**: Mengubah setiap elemen (misal: normalisasi).
- **`filter`**: Memilih elemen tertentu.
- **`shuffle`**: Mengacak data agar model tidak menghafal urutan.

In [ ]:
dataset = tf.data.Dataset.range(10)

# Map: kalikan dua, Filter: ambil yang > 10, Shuffle: acak dengan buffer
dataset = dataset.map(lambda x: x * 2) \
                 .filter(lambda x: x > 10) \
                 .shuffle(buffer_size=5, seed=42) \
                 .batch(3)

for item in dataset:
    print("Filtered & Shuffled Batch:", item.numpy())

## 3. Optimasi: Prefetching

**Prefetching** memastikan CPU menyiapkan batch $n+1$ saat GPU sedang mengerjakan batch $n$. Ini mengurangi waktu menganggur (idle) pada perangkat keras.

In [ ]:
# Gunakan tf.data.AUTOTUNE agar TensorFlow menentukan jumlah prefetch optimal
dataset = tf.data.Dataset.range(10).batch(3).prefetch(tf.data.AUTOTUNE)

## 4. Membaca Banyak File CSV (Interleave)

Jika data Anda terbagi dalam banyak file CSV, gunakan `interleave` untuk membaca baris dari banyak file secara paralel.

In [ ]:
def parse_line(line):
    # Contoh parsing CSV: 8 fitur float dan 1 label float
    defaults = [0.] * 8 + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defaults)
    return tf.stack(fields[:-1]), fields[-1]

# file_paths = ["file1.csv", "file2.csv", ...]
# dataset = tf.data.Dataset.list_files(file_paths)
# dataset = dataset.interleave(
#     lambda path: tf.data.TextLineDataset(path).skip(1),
#     cycle_length=5)
# dataset = dataset.map(parse_line, num_parallel_calls=tf.data.AUTOTUNE)

## 5. Format Biner TFRecord

**TFRecord** adalah format biner efisien yang menyimpan serialisasi Protocol Buffers. Sangat cepat untuk dibaca.

In [ ]:
# Menulis TFRecord
with tf.io.TFRecordWriter("data_latihan.tfrecord") as f:
    f.write(b"Data biner pertama")
    f.write(b"Data biner kedua")

# Membaca TFRecord
dataset = tf.data.TFRecordDataset(["data_latihan.tfrecord"])
for item in dataset:
    print(item.numpy())

## 6. Keras Preprocessing Layers

Memasukkan normalisasi dan encoding kategori langsung ke dalam model.

In [ ]:
from tensorflow.keras import layers

# Layer Normalisasi
norm_layer = layers.Normalization()
sample_data = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype="float32")
norm_layer.adapt(sample_data)

# Layer StringLookup (untuk kategori)
lookup = layers.StringLookup()
lookup.adapt(["Kucing", "Anjing", "Burung"])
print("Encoding 'Anjing':", lookup(tf.constant(["Anjing"])).numpy())

# Mengintegrasikan ke dalam model
model = tf.keras.models.Sequential([
    norm_layer,
    layers.Dense(10, activation="relu"),
    layers.Dense(1)
])

## Rangkuman Praktis

1. **Prefetching**: Gunakan di akhir setiap pipeline data.
2. **Parallelize Map**: Gunakan argumen `num_parallel_calls` agar proses CPU lebih cepat.
3. **TFRecord**: Gunakan format ini jika dataset Anda sangat masif untuk performa I/O maksimal.
4. **Normalization Adapt**: Pastikan memanggil `.adapt()` pada data sampel sebelum melatih model.